Import libraries

In [1]:
import os
import glob
import sys
from pyopenms import *
import pandas as pd
pd.set_option('future.no_silent_downcasting', True)

# `GNPSExport Workflow`

This workflow consists of 4 steps that generate a directory with all the files necessary to create a FBMN job at GNPS. (see https://ccms-ucsd.github.io/GNPSDocumentation/featurebasedmolecularnetworking-with-openms/) 

![GNPSExport.png](images/GNPSExport.png)

#### `1) Create a metadata tsv file for GNPS`

In [2]:
path= os.path.join("results", "GNPSexport")
isExist= os.path.exists(path)
if not isExist:
    os.mkdir(path)

# Create a metadata table from the list of mzML files 

metadata = pd.DataFrame()
metadata["filename"] = [file for file in os.listdir(os.path.join("data", "mzML")) if file.endswith(".mzML")]
metadata["ATTRIBUTE_MAPID"] = ["MAP" + str(i) for i in range(len(metadata))]
metadata['ATTRIBUTE_genomeID'] = metadata['filename'].str.extract(r'(NBC_?\d*)')
metadata['ATTRIBUTE_genomeIDMDNA'] = metadata['filename'].str.extract(r'(MDNAWGS?\d*|MDNA_WGS_?\d*)')
metadata['ATTRIBUTE_genomeID'] = metadata['ATTRIBUTE_genomeID'].fillna(metadata['ATTRIBUTE_genomeIDMDNA'])
metadata["ATTRIBUTE_media"] = metadata['filename'].str.extract(r'(ISP2|DNPM|FPY12|MA|soyM\d*)')
metadata["ATTRIBUTE_comment"] = metadata['ATTRIBUTE_genomeID'].astype(str) +"_" + metadata["ATTRIBUTE_media"].astype(str)
metadata=metadata.drop(columns = "ATTRIBUTE_genomeIDMDNA")
metadata.to_csv(os.path.join(path, "metadata.tsv"), sep='\t')
metadata

,filename,ATTRIBUTE_MAPID,ATTRIBUTE_genomeID,ATTRIBUTE_media,ATTRIBUTE_comment
0,Blank.mzML,MAP0,NaN,NaN,nan_nan
1,Control.mzML,MAP1,NaN,NaN,nan_nan
2,Pool.mzML,MAP2,NaN,NaN,nan_nan
3,Treatment.mzML,MAP3,NaN,NaN,nan_nan


Import aligned feature files and load them to a FeatureMap() file

#### `2) File Filter` 

Remove the features that do not have an MS2 pattern

###### Documentation: https://abibuilder.informatik.uni-tuebingen.de/archive/openms/Documentation/release/latest/html/TOPP_FileFilter.html

In [ ]:
path= os.path.join("results", "interim", "GNPSexport")
isExist= os.path.exists(path)
if not isExist:
    os.mkdir(path)

# Use Requantification consensus map if it exists, else use the one from Preprocessing
input_consensus = os.path.join("results", "interim", 
                               "Requantification" if os.path.exists(os.path.join("results", "interim", "Requantification")) else "Preprocessing",
                               "consensus.consensusXML")

cmap = ConsensusMap()
ConsensusXMLFile().load(input_consensus, cmap)
new_map= ConsensusMap(cmap)
new_map.clear(False)
for f in cmap:
    if f.getPeptideIdentifications() !=[]:
        new_map.push_back(f)

if new_map.empty():
    print("WARNING: No features with MS2 information. GNPSExport will output an empty file.")

Consensus_file= os.path.join(path ,'filtered' + ".consensusXML")
ConsensusXMLFile().store(Consensus_file, new_map)

blub


#### `3) MSMS clustering`

The algorithm creates an MGF file with only the MS2 information of all files (introduce mzml files with spaces between them)

###### Documentation: https://abibuilder.informatik.uni-tuebingen.de/archive/openms/Documentation/nightly/html/TOPP_GNPSExport.html

In [4]:
mzML_files = glob.glob(os.path.join("results", "interim", "mzML", "MapAligned_*.mzML"))

out_file = os.path.join("results", "GNPSexport", "MSMS.mgf")
spectra_clustering = GNPSMGFFile()
spectra_clustering.store(Consensus_file.encode(), [s.encode() for s in mzML_files], out_file.encode())

#### `4) FeatureQuantificationTable.txt`

Convert the consensusXML file to a txt file for GNPS

In [5]:
GNPSQuantificationFile().store(new_map, os.path.join("results", "GNPSexport", "FeatureQuantificationTable.txt"))

#### `5) IIMN (Ion Identity Molecular Networking)`:

https://ccms-ucsd.github.io/GNPSDocumentation/fbmn-iin/#iimn-networks-with-collapsed-ion-identity-edges

In [6]:
IonIdentityMolecularNetworking().annotateConsensusMap(new_map)
IonIdentityMolecularNetworking().writeSupplementaryPairTable(new_map, os.path.join("results", "GNPSexport", "SupplementaryPairTable.csv"))